# 🧠 QA Bot GUI with LangChain + Hugging Face

Upload a PDF and ask a question. Uses Chroma + sentence-transformers + Falcon LLM.

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
import gradio as gr

def build_qa_bot(pdf_path:str):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.split_documents(pages)
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    chroma_db = Chroma.from_documents(docs, embedding)
    retriever = chroma_db.as_retriever(search_kwargs={'k': 3})
    llm = HuggingFaceHub(repo_id='tiiuae/falcon-7b-instruct', model_kwargs={'temperature': 0.5, 'max_new_tokens': 256})
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

def answer_question(pdf_file_path, query):
    qa_bot = build_qa_bot(pdf_file_path)
    result = qa_bot({'query': query})
    return result['result']

demo = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.File(type='filepath', label='Upload a PDF'),
        gr.Textbox(lines=2, placeholder='Ask a question', label='Query')
    ],
    outputs='text',
    title='QA Bot with LangChain + Hugging Face LLM'
)
demo.launch(share=True)
